# client

> SDK to interact with the Agora-Fewsats Marketplace API

In [ ]:
#| default_exp client

In [ ]:
#| export


import httpx
from typing import Dict, List
from agora.config import *
from agora.crypto import *

from fastcore.utils import *


In [ ]:
# Test values

test_product_id = "678f61ec0205c9203bc07a45"
test_variant_id = 42884194664644

test_shipping_address = {
                "addressFirst": "123 Main St",
                "city": "New York",
                "state": "NY",
                "country": "US",
                "addressName": "Home",
                "zipCode": "10001"
            }
test_user = {
                "firstname": "John",
                "lastname": "Doe",
                "email": "john@example.com",
                "_id": "user123"
            }

In [ ]:
#| export


class Agora:
    """
    Client for the Agora-Fewsats Marketplace API.
    
    This client provides methods to interact with the Agora-Fewsats Marketplace API,
    including product search, cart management, and checkout functionality.
    """
    
    def __init__(self, base_url: str = "https://agora-backend.replit.app/api/v1", private_key: str = ''):
        """
        Initialize the Agora Marketplace client.
        
        Args:
            base_url: The base URL of the Agora-Fewsats Backend API.
            private_key: The private key of the Agora-Fewsats Backend API. If not provided,
                         a random UUID will be generated.
        """
        self.base_url = base_url

        cfg = get_cfg()

        if private_key: self.pk, self.pub = from_pk_hex(private_key) # if provided use the private key
        elif cfg.priv: self.pk, self.pub = from_pk_hex(cfg.priv) # if not provided use the private key from the config file
        else: 
            self.pk, self.pub = generate_keys()
            save_cfg({'priv': priv_key_hex(self.pk)})

        self.mongo_id = self.pub[:24] # agora uses mongoDB so the related user-objectId needs to be 24 characters

        self.client = httpx.Client(timeout=30.0)
        self.client.headers.update({"customuserid": self.mongo_id})

    def _make_request(self, method: str, endpoint: str, **kwargs) -> httpx.Response:
        """
        Make a request to the Agora-Fewsats Backend API.
        
        Args:
            method: The HTTP method to use.
            endpoint: The API endpoint to call.
            **kwargs: Additional arguments to pass to httpx.
            
        Returns:
            The response from the API.
        """
        url = f"{self.base_url}/{endpoint}"
        response = self.client.request(method, url, **kwargs)
        
        
        return response
        
    def search_products(self, query: str, count: int = 20, page: int = 1, 
                         price_min: int = 0, price_max: int = None, 
                         sort: str = None, order: str = None) -> Dict:
        """
        Search for products.
        
        Args:
            query: The search query.
            count: The number of products to return per page.
            page: The page number.
            price_min: The minimum price.
            price_max: The maximum price.
            sort: The sort field.
            order: The sort order.
            
        Returns:
            The search results.
        """
        params = {
            "q": query,
            "count": count,
            "page": page,
            "price_min": price_min
        }
        
        if price_max is not None:
            params["price_max"] = price_max
        if sort is not None:
            params["sort"] = sort
        if order is not None:
            params["order"] = order
        
        print(params)
        return self._make_request("GET", "search", params=params)


In [ ]:
a = Agora()
# a = Agora(base_url="http://localhost:8000/api/v1")
r = a.search_products("shirt", price_min=1, price_max=10)
r.status_code, r.json()

{'q': 'shirt', 'count': 20, 'page': 1, 'price_min': 1, 'price_max': 10}


(200,
 {'status': 'success',
  'Products': [{'_id': '667f0a9b90e7bb64d7c49e3e',
    'name': 'Shirt Clips Clear 1000/box',
    'storeName': 'Norton Supply',
    'brand': 'Norton Supply',
    'slug': 'shirt-clips-clear-1000-box-6495af47-2b24-43bc-aa47-1f219b3db1ef-1719601819634',
    'price': 7.36,
    'source': 'shopify',
    'images': ['https://cdn.shopify.com/s/files/1/1845/5117/products/shirt-clips-clear-1000box-592200.jpg?v=1703935439'],
    'url': 'https://nortonsupply.com/products/shirt-clips-clear-1000-box',
    'agoraScore': 100,
    'priceHistory': [{'price': 7.36,
      'date': '2024-06-28T19:14:09.406Z',
      '_id': '671370656ca0a1804a18989d'}],
    '_rankingScore': 0.6982421875,
    '_combinedScoreData': {'clipScore': 0.654296875,
     'cohereScore': 0.6982421875,
     'maxScore': 0.6982421875,
     'bestEmbedding': 'cohere',
     'position': 0},
    '_adjustedScore': 0.6982421875},
   {'_id': '6783b2eb9c59d2297121feec',
    'name': 'Milk Polo Shirt',
    'storeName': 'Milk

In [ ]:
p = r.json()["Products"][0]
p

{'_id': '667f0a9b90e7bb64d7c49e3e',
 'name': 'Shirt Clips Clear 1000/box',
 'storeName': 'Norton Supply',
 'brand': 'Norton Supply',
 'slug': 'shirt-clips-clear-1000-box-6495af47-2b24-43bc-aa47-1f219b3db1ef-1719601819634',
 'price': 7.36,
 'source': 'shopify',
 'images': ['https://cdn.shopify.com/s/files/1/1845/5117/products/shirt-clips-clear-1000box-592200.jpg?v=1703935439'],
 'url': 'https://nortonsupply.com/products/shirt-clips-clear-1000-box',
 'agoraScore': 100,
 'priceHistory': [{'price': 7.36,
   'date': '2024-06-28T19:14:09.406Z',
   '_id': '671370656ca0a1804a18989d'}],
 '_rankingScore': 0.6982421875,
 '_combinedScoreData': {'clipScore': 0.654296875,
  'cohereScore': 0.6982421875,
  'maxScore': 0.6982421875,
  'bestEmbedding': 'cohere',
  'position': 0},
 '_adjustedScore': 0.6982421875}

In [ ]:
#| export

@patch    
def get_product_detail(self: Agora, slug: str) -> Dict:
    """
    Get details for a specific product.
    
    Args:
        slug: The product slug.
        
    Returns:
        The product details.
    """
    params = {"slug": slug}
    return self._make_request("GET", "product-detail", params=params)


In [ ]:
r = a.get_product_detail(p['slug'])
r.json()

{'status': 'success',
 'product': {'_id': '667f0a9b90e7bb64d7c49e3e',
  'keywords': [],
  'name': 'Shirt Clips Clear 1000/box',
  'storeName': 'Norton Supply',
  'brand': 'Norton Supply',
  'tags': ['shipping_2to5'],
  'categories': ['Sports & Outdoors', 'Sports Apparel'],
  'description': '<div>In stock! Usually ships within 24 hours.</div><div>\nDescription<ul>\n<li>Our shirt clips, also known as ready-to-wear clothing clips, ensure that textiles stay in place in their packaging or during sales and presentations.</li>\n<li>They are widely used in the textile sector to keep clothing or fabric samples nicely folded.</li>\n<li>They are often used for shirts too, hence the name shirt clips. The ready-to-wear clothing clips are also frequently used in launderettes, dry cleaners and in the clothes hire sector.</li>\n<li>The advantage of our shirt clips is that they are transparent. They are therefore inconspicuous in textile displays and when used for other presentation purposes.</li>\n</u

In [ ]:
#| export

@patch
def get_cart(self: Agora) -> Dict:
    """
    Get the current user's cart.
    
    Returns:
        The cart details.
    """
    return self._make_request("GET", "cart")


In [ ]:
r = a.get_cart()
r.json()

{'id': 1,
 'customuserid': '1dd605e0a67f966bab10ceea',
 'created_at': 1742623590,
 'updated_at': 1742623592,
 'items': []}

In [ ]:
#| export
  
@patch
def add_to_cart(self: Agora, slug: str, product_id: str, variant_id: str = None, quantity: int = 1) -> Dict:
    """
    Add an item to the user's cart. Somem products do not have variants, in such cases use the product_id as variant_id too.
    
    Args:
        slug: The product slug.
        product_id: The product ID.
        variant_id: The product variant ID.
        quantity: The quantity to add.
        
    Returns:
        The updated cart.
    """
    item = {
        "slug": slug,
        "product_id": product_id,
        "variant_id": variant_id if variant_id else product_id,
        "quantity": quantity,
    }
    
    return self._make_request("POST", "cart/items", json=item)


In [ ]:
r = a.add_to_cart(p['slug'], p['_id'])
r.status_code, r.json()

(200,
 {'id': 1,
  'customuserid': '1dd605e0a67f966bab10ceea',
  'created_at': 1742623590,
  'updated_at': 1742623674,
  'items': [{'id': 1,
    'product_id': '667f0a9b90e7bb64d7c49e3e',
    'variant_id': '667f0a9b90e7bb64d7c49e3e',
    'quantity': 1,
    'title': 'Shirt Clips Clear 1000/box by Norton Supply',
    'description': 'Shirt Clips Clear 1000/box by Norton Supply from Norton Supply',
    'amount': 736,
    'currency': 'USD',
    'slug': 'shirt-clips-clear-1000-box-6495af47-2b24-43bc-aa47-1f219b3db1ef-1719601819634'}]})

In [ ]:
#| export

@patch
def update_cart_item(self: Agora, slug: str, product_id: str, variant_id: str, quantity: int) -> Dict:
    """
    Update the quantity of an item in the cart. Some products do not have variants, in such cases use the product_id as variant_id too.
    
    Args:
        slug: The product slug.
        product_id: The product ID.
        variant_id: The product variant ID.
        quantity: The new quantity.
        
    Returns:
        The updated cart.
    """
    params = {
        "slug": slug,
        "product_id": product_id,
        "variant_id": variant_id,
        "quantity": quantity
    }
    
    return self._make_request("PUT", "cart/items", params=params)
    


In [ ]:
r = a.update_cart_item(p['slug'], p['_id'], p['_id'], 1)
r.status_code, r.json()


(200,
 {'id': 1,
  'customuserid': '1dd605e0a67f966bab10ceea',
  'created_at': 1742623590,
  'updated_at': 1742623674,
  'items': [{'id': 1,
    'product_id': '667f0a9b90e7bb64d7c49e3e',
    'variant_id': '667f0a9b90e7bb64d7c49e3e',
    'quantity': 1,
    'title': 'Shirt Clips Clear 1000/box by Norton Supply',
    'description': 'Shirt Clips Clear 1000/box by Norton Supply from Norton Supply',
    'amount': 736,
    'currency': 'USD',
    'slug': 'shirt-clips-clear-1000-box-6495af47-2b24-43bc-aa47-1f219b3db1ef-1719601819634'}]})

In [ ]:
#| export

@patch
def clear_cart(self: Agora) -> Dict:
    """
    Clear all items from the cart.
    
    Returns:
        The response from the API.
    """
    return self._make_request("DELETE", "cart")

In [ ]:
r = a.clear_cart()
r.status_code, r.json()

(200, {'status': 'success', 'data': None, 'message': 'Cart cleared'})

In [ ]:
#| export

@patch
def buy_now(self: Agora, slug: str, product_id: str, variant_id: str, shipping_address: Dict,
            user: Dict, quantity: int = 1) -> Dict:
    """
    Purchase a product directly. Some products do not have variants, in such cases use the product_id as variant_id too.
    
    Args:
        slug: The product slug.
        product_id: The product ID.
        variant_id: The product variant ID.
        quantity: The quantity to purchase.
        shipping_address: The shipping address.
        user: The user information.
        
    Example:
        shipping_address = {
            "addressFirst": "123 Main St",
            "city": "New York",
            "state": "NY",
            "country": "US",
            "addressName": "Home",
            "zipCode": "10001"
        }
        
        user = {
            "firstname": "John",
            "lastname": "Doe",
            "email": "john@example.com",
        }
        
        a.buy_now(product_id, variant_id, shipping_address, user)
        
    Returns:
        The payment information.
    """

        
    request_data = {
        "slug": slug,
        "product_id": product_id,
        "variant_id": variant_id,
        "quantity": quantity,
        "shipping_address": shipping_address,
        "user": user
    }
    
    return self._make_request("POST", "buy-now", json=request_data)

In [ ]:
r = a.buy_now(p['slug'], p['_id'], p['_id'], test_shipping_address, test_user, quantity=1)
r.status_code, r.json()

(402,
 {'offers': [{'id': 'e26b48b4-fcf1-4bf2-a49b-b043d895016c',
    'amount': 736,
    'currency': 'USD',
    'description': 'Shirt Clips Clear 1000/box by Norton Supply from Norton Supply',
    'title': 'Shirt Clips Clear 1000/box by Norton Supply',
    'payment_methods': ['lightning', 'credit_card'],
    'type': 'one-off'}],
  'payment_context_token': '1750853d-880d-4496-8666-80ba78207678',
  'payment_request_url': 'https://api.fewsats.com/v0/l402/payment-request',
  'version': '0.2.2'})

In [ ]:
shipping_address = {
    "addressFirst": "Anselm Clave 1",
    "city": "Tordera", 
    "state": "Barcelona",
    "country": "ES",
    "addressName": "Home",
    "zipCode": "08490"
}

user = {
    "firstname": "Pol",
    "lastname": "Alvarez Vecino",
    "email": "pol.avms@gmail.com",
    "_id": "user123"
}

# Call buy_now with the parameters in the correct order
r = a.buy_now(
    slug="disappearing-ink-1-count-c2e1b357-1e86-46c7-bee0-328aefc5fe3f-1742527478059",
    product_id=7660653936801,
    variant_id=7660653936801,
    shipping_address=shipping_address,
    user=user,
    quantity=1  # Using default quantity of 1 since not specified
)
r.status_code, r.json()

(402,
 {'offers': [{'id': '47dc8cc6-06d8-403a-8949-4fba470ca6da',
    'amount': 199,
    'currency': 'USD',
    'description': 'Disappearing Ink (1 Count) by Set With Style from Set With Style',
    'title': 'Disappearing Ink (1 Count) by Set With Style',
    'payment_methods': ['lightning', 'credit_card'],
    'type': 'one-off'}],
  'payment_context_token': 'f8d3cbd0-c5c5-4a51-873a-f7b0a95598c8',
  'payment_request_url': 'https://api.fewsats.com/v0/l402/payment-request',
  'version': '0.2.2'})

In [ ]:
# test webhook 
offers = r.json()

from datetime import datetime, timezone

# Your existing data from buy_now
offer_id = offers['offers'][0]['id']
payment_context_token = offers['payment_context_token']
amount = offers['offers'][0]['amount']
currency = offers['offers'][0]['currency']

# Create a webhook payload
payload = {
    "offer_id": offer_id,
    "payment_context_token": payment_context_token,
    "amount": amount,
    "currency": currency,
    "status": "succeeded",  # Use 'succeeded' to simulate successful payment
    "timestamp": datetime.now(timezone.utc).isoformat()
}

# Send the request to your webhook endpoint
url = f"{a.base_url}/webhook/payment"  # Adjust if your server is running elsewhere
# url = "http://localhost:8000/api/v1/webhook/payment"  # Adjust if your server is running elsewhere
response = httpx.post(url, json=payload)


In [ ]:
#| export
   
@patch
def get_order(self: Agora, order_id: int) -> Dict:
    """
    Get details for a specific order.
    
    Args:
        order_id: The order ID.
        
    Returns:
        The order details.
    """
    return self._make_request("GET", f"orders/{order_id}")
    

@patch
def get_user_orders(self: Agora) -> List[Dict]:
    """
    Get all orders for the current user.
    
    Returns:
        A list of orders.
    """
    return self._make_request("GET", f"users/{self.customer_id}/orders")
    

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()